# Imports

In [1]:
import sys
import pymongo
from collections import defaultdict

import pandas as pd
import matplotlib.pyplot as plt

sys.path.append(r'../ScrapingTools')
from Sequential_Crawl import image_scrape
from TTB_scraping import TTB_Scraper

# Connect to databases and get desired TTBID's

##### Connect to databases

In [2]:
# Set up connection to mongodb
client = pymongo.MongoClient() # Connect to default client
db = client.TTB # Get a database (note: lazy evaluation)
TTB = db.TTB # collection for form data
TTB_COLORS = db.COLORS # collection for the label image data
TTB_IMG_META = db.IMG_META
TTB_IMG_SUP = db.IMG_SUP

##### Convert to pandas

In [3]:
df = pd.DataFrame(list(TTB.find()))

##### Generate state keys

In [4]:
# get list of all US states, convert to uppercase as that is what is used
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
states = [state.upper() for state in states]

us_state_abbrev = {'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY'}

# capitalized versions
abbrev_lookup=defaultdict(str)
for k, v in us_state_abbrev.items():
    abbrev_lookup[k.upper()] = v

##### Select out domestics

In [5]:
us_only = df.loc[df['OriginCode'].isin(states)]
us_only = us_only.loc[df['Status'] == 'APPROVED']
us_only['_id'].count()

38143

##### Get list of TTBIDs

In [6]:
us_only['TTBID'].count()

38143

We can use our log file to see where we stopped and pick up again from that point. Still unclear why we stopped prematurely...

In [7]:
us_only.loc[us_only['TTBID'] == '16137001000367']

,ApprovalDate,BrandName,Class/TypeCode,ContactInformation,FancifulName,ForSaleIn,Formula,GrapeVarietal(s),OriginCode,PlantRegistry/BasicPermit/BrewersNo(Other),...,Status,TTBID,TotalBottleCapacity,TypeofApplication,VendorCode,WineVintage,_id,recieve_code,recieve_date,seq_num
36124,06/09/2016,MARTIN RAY,TABLE WHITE WINE,MELISSAPRENTICE\nPhone Number:(707) 545-7593\n...,THE TOWER,,,N/A,CALIFORNIA,,...,APPROVED,16137001000367,,LABEL APPROVAL,20504,2015,16137001000367,001,05/16/2016,000367


In [8]:
len(us_only['TTBID'][36124:])

2019

In [9]:
#image_scrape(us_only['TTBID'][36124:])

# Find missing and rerun

##### Grab the data as a pandas table

In [10]:
#df_colors = pd.DataFrame(list(TTB_COLORS.find()))
df_img_meta = pd.DataFrame(list(TTB_IMG_META.find()))
#df_img_sup = pd.DataFrame(list(TTB_IMG_SUP.find()))

##### How many should we have?

In [11]:
len(us_only['TTBID'].unique())

38143

##### How many do we have?

In [12]:
len(df_img_meta['TTBID'].unique())

37327

##### Find missing TTBID's

In [13]:
missing = us_only[~us_only['TTBID'].isin(df_img_meta['TTBID'])]  # entries missing from _df_img_meta
missing = missing.loc[missing['recieve_code'] == '001']  # only entries that were submitted digitally
missing = missing['TTBID'].get_values()
len(missing)

9

##### RERUN ON MISSING

In [14]:
image_scrape(missing)

100%|██████████| 9/9 [00:06<00:00,  1.27it/s]


# Debug

In [91]:
missing[1]

'16112001000270'

In [105]:
missing[33]

'16137001000448'

In [92]:
scraper = TTB_Scraper(int(missing[0]))

In [93]:
meta, imgs = scraper.get_images()

In [94]:
meta

[]

In [35]:
plt.imshow(imgs[0])
plt.show()

IndexError: list index out of range